# Section 5: Model Registry & 本番活用

## 🎯 このセクションの目標
- Model Registryの概念と利点を理解する
- **Section 4で登録したv1モデルを確認**（Experiment Trackingとの連携）
- **バージョン管理（v1 → v2）** を体験する
- 登録したモデルを使って本番推論を実行する

## 💡 Model Registryとは？

機械学習モデルを一元管理するためのシステムです。

**主な利点:**
- 📦 **モデルのバージョン管理**（v1, v2, ...）
- 📝 メタデータ（メトリクス、説明）の管理
- 🔐 アクセス制御とガバナンス
- 🚀 SQL/Pythonからのシームレスな推論
- 🔄 **デフォルトバージョン**の設定

## 🔗 Section 4との連携

Section 4のExperiment Trackingで、最良モデル（v1）を`exp.log_model()`で登録しました。
このセクションでは、そのv1を確認し、さらにv2を追加してバージョン管理を体験します。

---
## 5.1 環境設定

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F

# ML関連（sklearn版を使用 - コンテナランタイム互換性のため）
import xgboost as xgb

# Model Registry
from snowflake.ml.registry import Registry

# 評価メトリクス（sklearn使用）
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# セッションの取得
session = get_active_session()

# 環境設定
session.use_database("MLOPS_HOL_DB")
session.use_schema("FEATURE_STORE")
session.use_warehouse("MLOPS_HOL_SQL_WH")

print("✅ ライブラリのインポート完了")

---
## 5.2 学習データの準備（v2用）

v2モデルの学習とv1との比較のため、データを準備します。
v1は既にSection 4でModel Registryに登録済みです。

In [ ]:
# 学習データの読み込み
training_data = session.table("MLOPS_HOL_DB.FEATURE_STORE.TRAINING_DATASET_V1")

# 特徴量の定義（Section 2-4と同じ）
FEATURE_COLS = [
    "DAYS_SINCE_LAST_ORDER",
    "TOTAL_ORDER_COUNT",
    "ORDER_COUNT_2024_H1",
    "TOTAL_ORDER_AMOUNT",
    "AVG_ORDER_AMOUNT",
    "TOTAL_AMOUNT_2024_H1",
    "RETURN_RATE"
]
LABEL_COL = "IS_CHURNED"
ID_COL = "CUSTOMER_ID"

# 訓練・テストデータの分割
model_data = training_data.select([ID_COL] + FEATURE_COLS + [LABEL_COL])
train_sf, test_sf = model_data.random_split([0.8, 0.2], seed=42)

# Pandas DataFrameに変換（sklearn用）
train_pd = train_sf.to_pandas()
test_pd = test_sf.to_pandas()

X_train = train_pd[FEATURE_COLS]
y_train = train_pd[LABEL_COL]
X_test = test_pd[FEATURE_COLS]
y_test = test_pd[LABEL_COL]

print(f"訓練データ: {len(train_pd):,} 件")
print(f"テストデータ: {len(test_pd):,} 件")

In [ ]:
# Section 4の実験結果を確認
print("=== Section 4 の実験結果 ===")
try:
    exp_results = session.table("MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_RESULTS")
    exp_results.select("RUN_NAME", "F1_SCORE", "PARAMS").show()
except:
    print("※ EXPERIMENT_RESULTSテーブルが見つかりません")
    print("  （Section 4を先に実行してください）")

# v1のパラメータ（Section 4で使用したもの）
# ※ v2との比較用に保持
v1_params = {
    "max_depth": 5,
    "learning_rate": 0.1,
    "n_estimators": 100,
    "scale_pos_weight": 2
}

print(f"\n📝 v1パラメータ（Section 4で登録済み）:")
print(f"   {v1_params}")

---
## 5.3 Section 4で登録したv1の確認

Section 4のExperiment Trackingで、`exp.log_model()`を使ってv1を登録しました。
ここでは、そのv1が正しく登録されているか確認します。

In [ ]:
# Model Registryのインスタンス作成
registry = Registry(
    session=session,
    database_name="MLOPS_HOL_DB",
    schema_name="MODEL_REGISTRY"
)

print("✅ Model Registry接続完了")

In [ ]:
# Section 4で登録したv1モデルの確認
MODEL_NAME = "CUSTOMER_CHURN_PREDICTOR"

print(f"=== 🔍 {MODEL_NAME} の確認 ===\n")

try:
    model = registry.get_model(MODEL_NAME)
    print(f"✅ モデル '{MODEL_NAME}' が見つかりました！")
    print(f"   （Section 4のExperiment Trackingで登録済み）\n")
    
    # バージョン一覧を表示
    print("=== 登録済みバージョン ===")
    model.show_versions()
    
except Exception as e:
    print(f"⚠️ モデルが見つかりません: {e}")
    print("   Section 4を先に実行してv1を登録してください。")

In [ ]:
# v1のメトリクスを取得（v2との比較用）
print("=== 📊 v1のメトリクス ===\n")

try:
    model = registry.get_model(MODEL_NAME)
    v1_version = model.version("v1")
    
    # メトリクスを取得
    v1_metrics_raw = v1_version.get_metric("*")
    
    # メトリクスを辞書形式に整形
    metrics = {}
    for metric_name, metric_value in v1_metrics_raw.items():
        metrics[metric_name] = metric_value
        print(f"   {metric_name}: {metric_value:.4f}")
    
    print(f"\n✅ v1メトリクス取得完了")
    
except Exception as e:
    print(f"⚠️ v1のメトリクス取得に失敗: {e}")
    print("   デフォルト値を使用します")
    # フォールバック用のデフォルト値
    metrics = {
        "accuracy": 0.85,
        "precision": 0.80,
        "recall": 0.75,
        "f1_score": 0.77
    }

# Lineage情報（v2登録時に使用）
FEATURE_STORE_VERSION = "v2"
TRAINING_DATASET = "TRAINING_DATASET_V1"

In [ ]:
# 登録されたモデルの確認
print("=== 登録済みモデル一覧 ===")
registry.show_models()

In [ ]:
# モデルのバージョン情報を確認
print(f"=== {MODEL_NAME} のバージョン情報 ===")
model = registry.get_model(MODEL_NAME)
model.show_versions()

---
## 5.4 バージョン管理の体験（v1 → v2）

Section 4で登録したv1に対して、**改良版v2を追加**してバージョン管理を体験します。

### v2 での改善点
- `n_estimators` を 100 → 200 に増加
- `max_depth` を 5 → 6 に増加
- より複雑なパターンを学習

In [ ]:
# v2用の改良パラメータ
v2_params = {
    "max_depth": 6,           # 5 → 6 に増加
    "learning_rate": 0.1,
    "n_estimators": 200,      # 150 → 200 に増加
    "scale_pos_weight": 3
}

print("🔄 v2モデルを学習中...")
print(f"   改善点: max_depth={v2_params['max_depth']}, n_estimators={v2_params['n_estimators']}")

v2_model = xgb.XGBClassifier(
    **v2_params,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

v2_model.fit(X_train, y_train)

# v2モデルの評価
v2_y_pred = v2_model.predict(X_test)

v2_metrics = {
    "accuracy": accuracy_score(y_test, v2_y_pred),
    "precision": precision_score(y_test, v2_y_pred),
    "recall": recall_score(y_test, v2_y_pred),
    "f1_score": f1_score(y_test, v2_y_pred)
}

print("✅ v2モデル学習完了")
print(f"   Accuracy:  {v2_metrics['accuracy']:.4f}")
print(f"   Precision: {v2_metrics['precision']:.4f}")
print(f"   Recall:    {v2_metrics['recall']:.4f}")
print(f"   F1 Score:  {v2_metrics['f1_score']:.4f}")

In [ ]:
# v2モデルをModel Registryに登録
VERSION_V2 = "v2"

# 既存のv2があれば削除（再実行対応）
import time as time_module

try:
    model = registry.get_model(MODEL_NAME)
    versions_before = [v.version_name for v in model.versions()]
    print(f"\n📋 削除前のバージョン一覧: {versions_before}")
    
    if VERSION_V2 in versions_before:
        print(f"🔄 既存の {VERSION_V2} を削除中...")
        model.delete_version(VERSION_V2)
        
        # 削除が反映されるまで待機（長めに）
        print(f"   削除反映を待機中（5秒）...")
        time_module.sleep(5)
        
        # 削除後の確認
        model = registry.get_model(MODEL_NAME)  # 再取得
        versions_after = [v.version_name for v in model.versions()]
        print(f"📋 削除後のバージョン一覧: {versions_after}")
        
        if VERSION_V2 not in versions_after:
            print(f"✅ {VERSION_V2} の削除を確認")
        else:
            print(f"⚠️ {VERSION_V2} がまだ存在します。再度削除を試みます...")
            model.delete_version(VERSION_V2)
            time_module.sleep(5)
    else:
        print(f"   {VERSION_V2} は存在しません（新規登録）")
        
except Exception as e:
    print(f"   （モデルが存在しないか、初回登録: {type(e).__name__}）")

# 🔗 Lineage情報（v2）
v2_comment = f"""チャーン予測モデル v2 - パラメータ改善版

【Lineage（系譜）情報】
- Feature Store: CUSTOMER_FEATURES {FEATURE_STORE_VERSION}
- 学習データ: {TRAINING_DATASET}
- 基となったバージョン: v1

【改善点】
- max_depth: 5 → 6
- n_estimators: 100 → 200

【パラメータ】
{v2_params}

【特徴量（{len(FEATURE_COLS)}個）】
{', '.join(FEATURE_COLS)}"""

print(f"🔄 v2モデルを登録中: {MODEL_NAME} ({VERSION_V2})")
print(f"   Feature Store: CUSTOMER_FEATURES {FEATURE_STORE_VERSION}")
print(f"   基バージョン: v1")

model_ref_v2 = registry.log_model(
    model=v2_model,
    model_name=MODEL_NAME,
    version_name=VERSION_V2,
    metrics=v2_metrics,
    sample_input_data=X_train.head(10),  # 入力形式のサンプル
    comment=v2_comment
)

print("✅ v2モデル登録完了！")
print(f"   モデル名: {MODEL_NAME}")
print(f"   バージョン: {VERSION_V2}")

In [ ]:
# バージョン一覧の確認
print(f"=== {MODEL_NAME} のバージョン一覧 ===")
model = registry.get_model(MODEL_NAME)
model.show_versions()

In [ ]:
# v1 vs v2 の比較
print("=" * 70)
print("📊 モデルバージョン比較")
print("=" * 70)
print(f"{'メトリクス':<15} {'v1':>12} {'v2':>12} {'差分':>12}")
print("-" * 70)

for metric_name in ['accuracy', 'precision', 'recall', 'f1_score']:
    v1_val = metrics[metric_name]
    v2_val = v2_metrics[metric_name]
    diff = v2_val - v1_val
    diff_str = f"{diff:+.4f}"
    print(f"{metric_name:<15} {v1_val:>12.4f} {v2_val:>12.4f} {diff_str:>12}")

print("=" * 70)

if v2_metrics['f1_score'] > metrics['f1_score']:
    print("✅ v2はF1スコアが改善しています！")
elif v2_metrics['f1_score'] < metrics['f1_score']:
    print("⚠️ v1の方がF1スコアが高いです")
else:
    print("➡️ F1スコアは同じです")

---
## 5.4.1 Lineage（系譜）テーブルの作成

**MLOpsの核心**: どのデータ → どの特徴量 → どの実験 → どのモデルか追跡可能にする

Model Registryのコメントに加え、**テーブル**としても記録することで、SQLで追跡可能にします。

In [ ]:
# Lineage（系譜）テーブルの作成
import datetime

lineage_records = [
    {
        "MODEL_NAME": MODEL_NAME,
        "MODEL_VERSION": "v1",
        "FEATURE_STORE_NAME": "CUSTOMER_FEATURES",
        "FEATURE_STORE_VERSION": FEATURE_STORE_VERSION,
        "TRAINING_DATASET": TRAINING_DATASET,
        "EXPERIMENT_NAME": "CHURN_PREDICTION_EXPERIMENT",
        "EXPERIMENT_RUN": "BestModel (Section 4)",  # Section 4で登録
        "PARAMETERS": str(v1_params),
        "ACCURACY": metrics.get("accuracy", 0.85),
        "F1_SCORE": metrics.get("f1_score", 0.77),
        "CREATED_AT": datetime.datetime.now().isoformat()
    },
    {
        "MODEL_NAME": MODEL_NAME,
        "MODEL_VERSION": "v2",
        "FEATURE_STORE_NAME": "CUSTOMER_FEATURES",
        "FEATURE_STORE_VERSION": FEATURE_STORE_VERSION,
        "TRAINING_DATASET": TRAINING_DATASET,
        "EXPERIMENT_NAME": "CHURN_PREDICTION_EXPERIMENT",
        "EXPERIMENT_RUN": "v2_improvement",
        "PARAMETERS": str(v2_params),
        "ACCURACY": v2_metrics["accuracy"],
        "F1_SCORE": v2_metrics["f1_score"],
        "CREATED_AT": datetime.datetime.now().isoformat()
    }
]

lineage_df = session.create_dataframe(lineage_records)
lineage_df.write.save_as_table(
    "MLOPS_HOL_DB.MODEL_REGISTRY.MODEL_LINEAGE",
    mode="overwrite"
)

print("✅ Lineage（系譜）テーブルを作成しました")
print("   テーブル: MLOPS_HOL_DB.MODEL_REGISTRY.MODEL_LINEAGE")


In [ ]:
# Lineageテーブルの確認
print("=== 📊 Model Lineage（系譜）===")
print("このテーブルで『どのモデル→どの特徴量→どの実験』を追跡できます\n")

lineage = session.table("MLOPS_HOL_DB.MODEL_REGISTRY.MODEL_LINEAGE")
lineage.select(
    "MODEL_VERSION",
    "FEATURE_STORE_VERSION", 
    "EXPERIMENT_RUN",
    "F1_SCORE"
).show()


### 💡 Lineageテーブルの活用例

```sql
-- Q: モデルv2はどの特徴量を使っている？
SELECT FEATURE_STORE_NAME, FEATURE_STORE_VERSION, TRAINING_DATASET
FROM MODEL_REGISTRY.MODEL_LINEAGE
WHERE MODEL_VERSION = 'v2';

-- Q: 特徴量v2を使っているモデルは？
SELECT MODEL_NAME, MODEL_VERSION, F1_SCORE
FROM MODEL_REGISTRY.MODEL_LINEAGE
WHERE FEATURE_STORE_VERSION = 'v2';

-- Q: 実験Baselineの結果からデプロイされたモデルは？
SELECT MODEL_NAME, MODEL_VERSION
FROM MODEL_REGISTRY.MODEL_LINEAGE
WHERE EXPERIMENT_RUN = 'Baseline';
```

### 🔗 追跡可能性（Traceability）の確保

| 確認したいこと | 参照先 |
|--------------|--------|
| モデル → 特徴量 | MODEL_LINEAGE.FEATURE_STORE_VERSION |
| モデル → 実験 | MODEL_LINEAGE.EXPERIMENT_RUN |
| モデル → パラメータ | MODEL_LINEAGE.PARAMETERS |
| モデル → メトリクス | MODEL_REGISTRY の Versions |
| 詳細なコメント | Model Registry UI のコメント欄 |


---
## 5.5 v2モデルを使った推論

v2モデルを使って、テストデータに対してチャーンスコアを計算します。

> 💡 **注**: sklearn版モデルのModel Registry推論（`model.run()`）はSnowpark Container Servicesが必要です。
> ここでは学習済みの `v2_model` を直接使用します。

In [ ]:
# v2モデルでテストデータを推論（sklearn版モデルを直接使用）
test_predictions = v2_model.predict(X_test)
test_proba = v2_model.predict_proba(X_test)[:, 1]

# 結果をDataFrameに追加
inference_pd = test_pd.copy()
inference_pd["PREDICTED_CHURN"] = test_predictions
inference_pd["CHURN_PROBABILITY"] = test_proba

# Snowparkデータフレームに変換
inference_result = session.create_dataframe(inference_pd)

print(f"✅ 推論完了（v2モデルを使用）")
print(f"   予測レコード数: {len(inference_pd):,}")
inference_result.select(ID_COL, "PREDICTED_CHURN", "CHURN_PROBABILITY").show(10)

---
## 5.6 チャーンリスク顧客リストの作成

In [ ]:
# チャーンリスク高の顧客を抽出
high_risk_customers = inference_result.filter(
    F.col("PREDICTED_CHURN") == 1
)

print(f"🎯 チャーンリスク高の顧客数: {high_risk_customers.count():,}")

In [ ]:
# 顧客情報と結合
customer_info = session.table("MLOPS_HOL_DB.PREP_DATA.CUSTOMERS")

target_list = high_risk_customers.join(
    customer_info,
    on="CUSTOMER_ID",
    how="inner"
).select(
    F.col("CUSTOMER_ID"),
    F.col("SEGMENT"),
    F.col("REGION"),
    F.col("PREDICTED_CHURN").alias("CHURN_RISK")
)

print("=== チャーンリスク高の顧客リスト（サンプル） ===")
target_list.show(10)

In [ ]:
# ターゲットリストをテーブルに保存
target_list.write.save_as_table(
    "MLOPS_HOL_DB.ANALYTICS.CHURN_RISK_CUSTOMERS",
    mode="overwrite"
)

print("✅ チャーンリスク顧客リストを保存しました")
print("   テーブル名: MLOPS_HOL_DB.ANALYTICS.CHURN_RISK_CUSTOMERS")

---
## 5.7 ビジネスアウトカムの確認

In [ ]:
# サマリーレポート
total_test_customers = len(test_pd)  # test_pd は Cell 4 で定義
high_risk_count = high_risk_customers.count()
churn_rate = high_risk_count / total_test_customers * 100

print("=" * 60)
print("🎯 ビジネスアウトカム サマリー")
print("=" * 60)
print(f"  テスト顧客数:            {total_test_customers:,} 人")
print(f"  チャーンリスク高顧客数:  {high_risk_count:,} 人")
print(f"  予測チャーン率:          {churn_rate:.1f}%")
print("-" * 60)
print(f"  📧 リテンション施策対象: {high_risk_count:,} 人")
print("=" * 60)

In [ ]:
# セグメント別のチャーンリスク分布
print("=== セグメント別チャーンリスク分布 ===")
target_list.group_by("SEGMENT") \
    .count() \
    .order_by(F.col("COUNT").desc()) \
    .show()

In [ ]:
# 地域別のチャーンリスク分布
print("=== 地域別チャーンリスク分布 ===")
target_list.group_by("REGION") \
    .count() \
    .order_by(F.col("COUNT").desc()) \
    .show()

---
## 5.8 SQLからのモデル呼び出し

Model Registryに登録されたモデルは、**SQLから直接呼び出す**ことができます。

In [ ]:
print("=== SQLでのモデル呼び出し例 ===")
sql_example = """
-- 新しい顧客データに対してチャーン予測を実行
SELECT 
    CUSTOMER_ID,
    MLOPS_HOL_DB.MODEL_REGISTRY.CUSTOMER_CHURN_PREDICTOR!PREDICT(
        DAYS_SINCE_LAST_ORDER,
        TOTAL_ORDER_COUNT,
        ORDER_COUNT_2024_H1,
        TOTAL_ORDER_AMOUNT,
        AVG_ORDER_AMOUNT,
        TOTAL_AMOUNT_2024_H1,
        RETURN_RATE
    ) AS CHURN_PREDICTION
FROM CUSTOMER_FEATURES_VIEW
WHERE CHURN_PREDICTION = 1;
"""
print(sql_example)

---
## ✅ Section 5 完了！ハンズオン終了！ 🎉

### 学んだこと（全セクション）

| セクション | 内容 |
|-----------|------|
| Section 1 | データ探索、**チャーンの定義・ラベル作成** |
| Section 2 | Feature Store（Entity, FeatureView, バージョン管理） |
| Section 3 | XGBoostでのモデル学習、CV、SHAP |
| Section 4 | Experiment Tracking（実験管理、**Model Registryへv1登録**） |
| Section 5 | Model Registry（**v1確認**、v2追加、バージョン管理、本番推論） |

### 🔗 MLOpsの一気通貫フロー

```
Section 4: Experiment Tracking
    ↓ exp.log_model() で連携
Section 5: Model Registry
    ↓ 実験ラン ↔ モデルバージョン がリンク
Snowsight: 「モデルバージョン」に実験情報が表示！
```

### 構築したもの

- ✅ **CHURN_LABELS**: 定義・作成したチャーンラベル
- ✅ **Feature Store**: 7個の特徴量（RFM + 行動）、v1→v2バージョン管理
- ✅ **実験管理**: 3つのモデルを比較し、最適なものを選択
- ✅ **Model Registry**: 本番モデルを登録、v1→v2バージョン管理
- ✅ **Experiment ↔ Model連携**: 実験からモデルへの追跡可能性
- ✅ **チャーンリスク顧客リスト**: リテンション施策の対象リスト

### バージョン管理の比較

| 機能 | Feature Store | Model Registry |
|------|--------------|----------------|
| バージョン | v1, v2, ... | v1, v2, ... |
| 改善方法 | 特徴量追加・修正 | パラメータ・アルゴリズム変更 |
| 比較 | 特徴量の差分確認 | メトリクスの比較 |
| ロールバック | 過去バージョンを参照 | 過去バージョンで推論 |

### 次のステップ（発展編）

1. **ハイパーパラメータ最適化**: `Tuner` を使った自動チューニング
2. **モデルモニタリング**: データドリフトの検出
3. **リアルタイム推論**: Snowpark Container Services
4. **CI/CD連携**: GitHub Actions + Snowflake CLI

---
## 📚 参考リンク

- [Snowflake ML Overview](https://docs.snowflake.com/en/developer-guide/snowflake-ml/overview)
- [Snowflake Feature Store](https://docs.snowflake.com/en/developer-guide/snowflake-ml/feature-store/overview)
- [Snowflake Model Registry](https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/overview)
- [Snowflake ML Experiments](https://docs.snowflake.com/en/developer-guide/snowflake-ml/experiments)
- [Snowpark ML Modeling](https://docs.snowflake.com/en/developer-guide/snowflake-ml/modeling)